<a href="https://colab.research.google.com/github/RonanKMcGovern/qlora-functions/blob/fixPrompt/Llama_2_7b_chat_hf_sharded_bf16_QLoRA_for_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *About*

Forked from: https://colab.research.google.com/drive/1FxlUb_H6Xirhkx4RszAgHeb2uDW7oKIH by https://www.youtube.com/@vrsen

Authored by: RonanKMcGovern of https://research.trelis.com and https://huggingface.co/Trelis

# Install

In [2]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 997.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [ ]:
model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB" # sharded model by RonanKMcGovern
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

In [ ]:
print(dir(tokenizer))


In [ ]:
print("Current EOS token:", tokenizer.eos_token)


In [ ]:
if "[/INST]" in tokenizer.get_vocab():
    print("[/INST] is in the vocabulary.")
else:
    print("[/INST] is not in the vocabulary.")


In [ ]:
import json


In [ ]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# The JSON object to be stringified and tokenized
json_obj = {
    "name": "search_arxiv",
    "description": "Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query string",
            }
        },
        "required": ["query"],
    },
}

# Stringify the JSON object
json_string = json.dumps(json_obj)

# Tokenize the stringified JSON object
tokens = tokenizer.encode(json_string)

# Count the number of tokens
token_count = len(tokens)

# Fetch BOS and EOS tokens
BOS_token_id = tokenizer.bos_token_id
EOS_token_id = tokenizer.eos_token_id

# Check and print BOS and EOS tokens
print(f"Beginning of the sequence: {tokens[0]} (BOS token: {BOS_token_id})")
print(f"End of the sequence: {tokens[-1]} (EOS token: {EOS_token_id})")

print(f"The number of tokens in the stringified JSON object is: {token_count}")

In [ ]:
print(f"Model's maximum length: {tokenizer.model_max_length}")

In [ ]:
print(model.config)


In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

In [ ]:
print(model.state_dict().keys())

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


In [ ]:
### Original LoRa config - DO NOT RUN

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Prepare Data

In [ ]:
from datasets import load_dataset

data = load_dataset("Trelis/function_calling", "train")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['systemPrompt', 'userPrompt', 'assistantResponse'],
        num_rows: 11
    })
    test: Dataset({
        features: ['systemPrompt', 'userPrompt', 'assistantResponse'],
        num_rows: 5
    })
})


### Training Data

In [ ]:
# Define the roles and markers
B_SYS, E_SYS = "<<SYS>>", "<</SYS>>"
B_INST, E_INST = "[INST]", "[/INST]"

# Define system, user, and assistant roles
SYSTEM_ROLE = "system"
USER_ROLE = "user"
ASSISTANT_ROLE = "assistant"

# Create the formatted text with the correct roles for each part of the dialogue
train_dataset = data['train'].map(
    lambda x: {
        "input_text": "".join([
            f"{B_SYS}{x['systemPrompt'].strip()}{E_SYS}",
            f"{B_INST}{x['userPrompt'].strip()}{E_INST}",
            f"{B_INST}{x['assistantResponse'].strip()}{E_INST}",
        ])
    }
)

# Tokenize the datasets
train_encodings = tokenizer([dialogue["input_text"] for dialogue in train_dataset], truncation=True, padding=True, max_length=512, return_tensors='pt')

In [ ]:
print(train_encodings['input_ids'].size())


torch.Size([11, 460])


### Test Data

In [ ]:
# Define the roles and markers
B_SYS, E_SYS = "<<SYS>>", "<</SYS>>"
B_INST, E_INST = "[INST]", "[/INST]"

# Define system, user, and assistant roles
SYSTEM_ROLE = "system"
USER_ROLE = "user"
ASSISTANT_ROLE = "assistant"

# Create the formatted text with the correct roles for each part of the dialogue
test_dataset = data['test'].map(
    lambda x: {
        "input_text": "".join([
            f"{B_SYS}{x['systemPrompt'].strip()}{E_SYS}",
            f"{B_INST}{x['userPrompt'].strip()}{E_INST}",
            f"{B_INST}{x['assistantResponse'].strip()}{E_INST}",
        ])
    }
)

# Tokenize the datasets
test_encodings = tokenizer([dialogue["input_text"] for dialogue in test_dataset], truncation=True, padding=True, max_length=512, return_tensors='pt')

In [ ]:
print(test_encodings['input_ids'].size())

torch.Size([5, 444])


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

In [ ]:
# Convert the encodings to PyTorch datasets
test_dataset = TextDataset(test_encodings)

In [ ]:
import textwrap
wrapper = textwrap.TextWrapper(width=80)

In [1]:
import re  # import regular expressions module

In [ ]:
def generate(index):
    system_prompt = data['test'][index]['systemPrompt']
    user_prompt = data['test'][index]['userPrompt']
    correct_answer = data['test'][index]['assistantResponse']

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    # Construct prompt directly
    prompt = f"{B_INST}{B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()}{E_INST}"

    print("Prompt:")
    print(prompt)

    # Use the tokenizer to encode the prompt, set return_tensors="pt" to return PyTorch tensors
    # Use bos=True to include BOS token, and eos=False because we want to generate the assistant's response
    encoding = tokenizer.encode(prompt, return_tensors="pt", bos=True, eos=False).to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=200, do_sample=True, temperature=0.01, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    print()

    # Subtract the length of input_ids from output to get only the model's response
    output_text = tokenizer.decode(output[0, len(encoding):], skip_special_tokens=True)
    output_text = re.sub('\n+', '\n', output_text)  # remove excessive newline characters

    print("Generated Assistant Response:")
    print(output_text)

    print()

    print("Correct Assistant Response:")
    print(correct_answer)

    print()

In [ ]:
generate(1)

Prompt:
<<SYS>>
You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.\",\"parameters\":{\"typ

KeyboardInterrupt: ignored

# Training

In [ ]:
# Define a function to print some samples
def print_samples(dataset, tokenizer, num_samples=5):
    for i, sample in enumerate(dataset):
        if i >= num_samples:  # Stop after printing the desired number of samples
            break
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        labels = sample['labels']

        # Decode the input and the label
        decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
        decoded_label = tokenizer.decode(labels, skip_special_tokens=True)

        print(f"Sample {i+1}:")
        print(f"Input: {decoded_input}")
        print(f"Label: {decoded_label}")
        print("----------------------")

# Use the function to print some samples from your train_dataset
print_samples(train_dataset, tokenizer)

Sample 1:
Input: <<SYS>>You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.\",\"parameters\

<ipython-input-135-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
#original training script

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, #turn on the eval dataset for comparisons
    args=transformers.TrainingArguments(
        num_train_epochs=2,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_ratio=0.05,
        max_steps=25,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

<ipython-input-135-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
1,0.329600
2,0.258000
3,0.218100
4,0.205400
5,0.201900
6,0.206800
7,0.178700
8,0.121500
9,0.155700
10,0.100100


TrainOutput(global_step=25, training_loss=0.11425744011998176, metrics={'train_runtime': 86.8758, 'train_samples_per_second': 0.576, 'train_steps_per_second': 0.288, 'total_flos': 429901559562240.0, 'train_loss': 0.11425744011998176, 'epoch': 4.17})

# Example After Fine Tuning

In [ ]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): LlamaForCausalLM(
              (model): LlamaModel(
                (embed_tokens): Embedding(32000, 4096, padding_idx=0)
                (layers): ModuleList(
                  (0-31): 32 x LlamaDecoderLayer(
                    (self_attn): LlamaAttention(
                      (q_proj): Linear4bit(
                        in_features=4096, out_features=4096, bias=False
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.05, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=4096, out_features=16, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (default): Linear(in_features=16,

In [ ]:
generate(3)

Prompt:
<<SYS>>
You are a helpful research assistant. The following functions, if any, are available for you to fetch further data to answer user questions: {\"name\":\"search_bing\",\"description\":\"Search the web for content on Bing. This allows users to search online/the internet/the web for content.\",\"parameters\":{\"type\":\"object\",\"properties\":{\"query\":{\"type\":\"string\",\"description\":\"The search query string\"}},\"required\":[\"query\"]}}{\"name\":\"get_current_weather\",\"description\":\"Get the current weather in a given location\",\"parameters\":{\"type\":\"object\",\"properties\":{\"location\":{\"type\":\"string\",\"description\":\"The city and state, e.g. San Francisco, CA\"},\"unit\":{\"type\":\"string\",\"enum\":[\"celsius\",\"fahrenheit\"]}},\"required\":[\"location\"]}}{\"name\":\"search_arxiv\",\"description\":\"Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.\",\"parameters\":{\"typ

# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.save_pretrained("falcon-7b-instruct-ft-adapters", push_to_hub=True) # save LORA adapters
model.push_to_hub("falcon-7b-instruct-ft-adapters")

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft-adapters/commit/4451b70e404c45ffc7d27ce7da4313d0add26982', commit_message='Upload model', commit_description='', oid='4451b70e404c45ffc7d27ce7da4313d0add26982', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, PretrainedConfig
import torch

# reload the base model (you might need pro subscription for this)
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct", device_map={"":0}, trust_remote_code=True, torch_dtype=torch.float16)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    "vrsen/falcon-7b-instruct-ft-adapters",
)

In [ ]:
model = model.merge_and_unload() # merge adapters with

In [ ]:
model.push_to_hub("falcon-7b-instruct-ft")

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/93f6d84447abd80fff5ff062c81dc86b0ac72e82', commit_message='Upload RWForCausalLM', commit_description='', oid='93f6d84447abd80fff5ff062c81dc86b0ac72e82', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.push_to_hub("falcon-7b-instruct-ft", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/ae7eca5a54426134be2babe2239380487dad3306', commit_message='Upload tokenizer', commit_description='', oid='ae7eca5a54426134be2babe2239380487dad3306', pr_url=None, pr_revision=None, pr_num=None)